In [1]:
import pandas as pd
from collections import Counter
import re
import math
df = pd.read_csv("../Data/2019/2019_TFIDF_DBSCAN_CV_eps8_mp20.csv", encoding='utf-8-sig')

In [2]:
df = df.sort_values(by='labels', ascending=True)
df.tail()

,N_list,category,date,headline,img,press,text,url,topics,labels,cnt
3091,"[['서울시', '의회', '우리공화당', '광화문광장', '천막', '서울시', ...","사회,지역",2019-06-28,서울시의회 “우리공화당 불법 천막 규탄…엄중히 대처 할 것” 촉구 결의안 가결,['//img.segye.com/content/image/2019/06/28/201...,세계일보,서울시의회는 우리공화당(구 대한애국당)의 광화문광장 천막에 대해 서울시가 엄정한 법...,http://www.segye.com/content/html/2019/06/28/2...,"우리공화당, 천막, 철거, 광화문광장, 집행, 설치, 불법, 시의회, 공권력, 결의안",59,61
3090,"[['대한애국당', '기습', '광화문광장', '천막', '동', '설치'], ['...","사회,국제",2019-06-25,"공화당, 천막 철거 3시간 만에 재설치…서울시 ""단호 대응""",['https://pds.joins.com/news/component/htmlpho...,중앙일보,대한애국당(현 우리공화당)이 기습적으로 광화문광장에 천막 3동을 다시 설치했다. 서...,https://news.joins.com/article/olink/23098468,"천막, 철거, 서울시, 공화당, 설치, 용역, 우리공화당, 계획, 대한애국당, 광화문광장",59,61
3089,"[['우리공화당', '행정', '집행', '서울시', '근본', '대책', '경찰'...","사회,국제",2019-06-30,'광화문 천막 사태' 재발 우려…트럼프 떠난 뒤 다시 칠 가능성,['//img.segye.com/content/image/2019/06/30/201...,세계일보,"우리공화당, 세 번째 설치땐 행정대집행 불가피서울시 ""근본대책 절실""경찰, 시설물보...",http://www.segye.com/content/html/2019/06/30/2...,"천막, 우리공화당, 광화문광장, 서울시, 행정, 집회, 집행, 철거, 경찰, 광장",59,61
3087,"[['강제', '철거', '대한애국당', '기습', '설치', '광화문광장', '농...","사회,정치",2019-05-13,서울시 “애국당 천막 세월호와 달라”… 강제철거는 14일부터 가능,['//img.segye.com/content/image/2019/05/13/201...,세계일보,강제철거는 14일부터 가능대한애국당이 기습적으로 설치한 광화문광장 농성 천막에 대해...,http://www.segye.com/content/html/2019/05/13/2...,"철거, 천막, 서울시, 자진, 설치, 대한애국당, 불법, 강제, 상금, 광화문광장",59,61
3123,"[['우리공화당', '광화문광장', '불법', '설치', '천막', '자진', '철...","국제,정치",2019-06-28,"“트럼프를 질서있게 환영하기 위해” 공화당, 광화문광장 천막 철거",['http://image.kmib.co.kr/online_image/2019/06...,국민일보,우리공화당이 광화문광장에 불법으로 설치한 천막을 28일 자진 철거했다. 지난달 10...,http://news.kmib.co.kr/article/view.asp?arcid=...,"천막, 우리공화당, 철거, 광화문광장, 자진, 서울시, 설치, 조원진, 환영, 트럼프",59,61


In [4]:
def npmi(a, b, n, z):
    x = math.log10((n*z)/(a*b))
    y = math.log10(n/z)
    return x / y

In [24]:
# 뉴스 기사별 PMI 체크
merge_tokens = []
docs = []
for article in df['N_list']:
    doc = []
    com = []
    context = []
    merge_token = []
    merge_token2 = []

    # str -> list
    for line in article.split('], ['):
        arr = re.sub("[\[,'\]]", '', line).split(' ')
        doc.append(arr)
        context += arr

    #2 PMI 적용
    for line in doc:
        com += list(zip(line, line[1:]))
    n = len(context)
    cnt = Counter(context)
    nmi = {}
    z = Counter(com)
    for pair in com:
        pmi = npmi(cnt[pair[0]], cnt[pair[1]], n, z[pair])
        if -1 < pmi < 1:
            nmi[pair] = pmi
    result = sorted(nmi.items(), key = lambda item : item[1], reverse=True)
    merge_token = [key[0]+"_"+key[1] for key, item in result if item >= 0.91]
    if len(merge_token) > 0:
        merge_tokens+=merge_token
    # 토큰을 합치는 내용
    for token in merge_token:
        for content in doc:
            for i in range(len(content)-1):
                cmp_str = '_'.join(content[i:i+2])
                if cmp_str == token:
                    del content[i:i+2]
                    content.insert(i, token)
    docs.append(doc)
z = Counter(merge_tokens).most_common()
print(z)

[('김_위원장', 309), ('정상_회담', 209), ('트럼프_대통령', 176), ('북_미', 96), ('기자_회견', 93), ('김_차관', 76), ('성_매매', 62), ('메이_총리', 51), ('대북_제재', 42), ('합의_안', 38), ('증거_인멸', 38), ('조_회장', 37), ('개학_연기', 35), ('국민_연금', 34), ('문_대통령', 31), ('영업_이익', 31), ('유_대표', 31), ('분식_회계', 29), ('국무위원_장', 28), ('박_대표', 26), ('구속_영장', 26), ('허블레아니_호', 24), ('기금_운용', 23), ('제재_완화', 22), ('차_북미', 22), ('상응_조치', 21), ('실무_협상', 21), ('브렉_시트', 21), ('아베_총리', 21), ('국가_비상사태', 20), ('김_장관', 20), ('국민_투표', 20), ('비례_대표', 20), ('산하_기관', 20), ('북미_정상', 19), ('수탁자_책임', 19), ('패스트_트랙', 19), ('탄력_근로', 19), ('최저_임금', 18), ('불법_촬영', 18), ('특별_대표', 17), ('공시_가격', 17), ('부장_판사', 17), ('이하_징역', 17), ('영장_청구', 17), ('매매_알선', 17), ('비상사태_선포', 16), ('양_대법원장', 16), ('사업_지원', 16), ('정_사장', 16), ('정상_회의', 15), ('사내_이사', 15), ('김_기획관', 15), ('행정_집행', 15), ('제재_해제', 14), ('윤_총경', 14), ('창원_성산', 14), ('주거_침입', 14), ('극동_연방', 13), ('단위_기간', 13), ('보고서_채택', 13), ('수입_금지', 13), ('전술_유도', 13), ('강간_미수', 13), ('전체_회의', 12), ('여야_당', 12), ('영장_신

In [24]:
df['N_list'] = docs
df['tokens'] = merge_tokens
df.to_csv("./Data/2019_DBSCAN.csv", encoding="utf-8-sig", index=False)

In [6]:
# 클러스터별 pmi 체크
cluster = {}
for i in range(df['labels'].max()+1):
    print(i,": ================")
    word_list = []
    com=[]
    for article in df[df['labels'] == i]['N_list']:
        # str -> list
        for line in article.split('], ['):
            arr = re.sub("[\[,'\]]", '', line).split(' ')
            word_list += arr
            com += list(zip(arr, arr[1:]))
    print(len(word_list))
    print(len(com))
    n = len(word_list)
    cnt = Counter(word_list)
    nmi = {}
    z = Counter(com)
    for pair in com:
        pmi = npmi(cnt[pair[0]], cnt[pair[1]], n, z[pair])
        nmi[pair] = pmi
    result = sorted(nmi.items(), key = lambda item : item[1], reverse=True)
    merge_token = [key[0]+"_"+key[1] for key, item in result if item >= 0.9]
    # merge_token = [(key, item) for key, item in result if item >= 0.9]
    print(merge_token)
    cluster[i] = merge_token

0 : ================
310860
289296
['롱코트_중절모', '보리스_옐친', '만두_펠메니', '수프_보르', '고공_행진', '엔화_급등', '하나은행_딜링', '딜러_전광판', '앤드류_틸튼', '웰스_매니지먼트', '장착_구축함', '구축함_프레', '거침_오름세', '서구_열강', '상공_회의소', '체코_헝가리', '헝가리_취약', '고온승압용합_금강', '금강_무봉강관', '거품_뇌관', '클라우드_컴퓨팅', '유전자_변형', '니니스_퇴', '뚜벅뚜_벅', '스쿼크_박스', '레이시온_합병', '암스트롱_컬럼비아', '러시안레일_웨이즈', '삼각관계_조망', '모독_저질', '장금_철로', '김준형_한동', '욱일승천기_자위대', '자위대_호위함', '갈라파고스_블랙홀', '자녀_직장', '김현정_뉴스쇼', '카지_아니스', '김병연_서울대', '배병우_논설', '평창_동계', '동계_올림픽', '평론가_천다오인', '삼성_현대차', '현대차_SK', 'SK_LG', '클로드_융커', '야누코_비치', '오울_렛', '로젠_버거', '앨리슨_후커', '데니스_와일더', '모리슨_호주', '펜실베이니아_노스캐롤라이나', '개헌_추진하는아베', '국군_유엔군', '최태원_SK그룹', '구광모_LG그룹', '신동빈_롯데그룹', '야마_모토', '벨라_비아', '비아_예비역', '예비역_육군', '육군_하사', '나이키_아디다스', '소임_정열', '싱크_탱크', '인텔_마이크론', '재난_질병', '한강_하구', '외평_가산', '연방준비제도_제롬', '출연금_엔', '항공모함_랴오닝함', '모항_산둥', '빗장_선례', '친척_스파이', '로즈_가든', '곽_혜진', '흉내_넘사', '홈_그라운드', '중국국제방송_차이나데일', '번드르르_치장', '관객_감흥', '북한군_묘비', '티나_터너', '학번_아산', '아산_서원', '기선_제압', '발렌티_마트비옌', '앤_콘웨이', '아메리칸_컨서버티', '필_족자', '사자성

In [20]:
# 뉴스 기사별 N_GRAM 체크
merge_tokens = []
docs = []
for article in df['N_list']:
    doc = []
    com = []
    context = []
    merge_token = []

    # str -> list
    for line in article.split('], ['):
        arr = re.sub("[\[,'\]]", '', line).split(' ')
        doc.append(arr)
        com += list(zip(arr, arr[1:]))

    #2 N_GRAM 체크
    z = Counter(com).most_common()
    for item, value in z:
        if value > 3:
            merge_token.append(item[0]+'_'+item[1])
    if len(merge_token) > 0:
        merge_tokens += merge_token

    # 토큰을 합치는 내용
    for token in merge_token:
        for content in doc:
            for i in range(len(content)-1):
                cmp_str = '_'.join(content[i:i+2])
                if cmp_str == token:
                    del content[i:i+2]
                    content.insert(i, token)
    docs.append(doc)
z = Counter(merge_tokens).most_common()
print(z)

[('트럼프_대통령', 871), ('정상_회담', 795), ('김_위원장', 458), ('브렉_시트', 224), ('북_미', 215), ('문_대통령', 181), ('메이_총리', 158), ('미_정상', 142), ('합의_안', 126), ('북미_정상', 125), ('구속_영장', 119), ('성_매매', 100), ('국민_연금', 93), ('개학_연기', 90), ('조_회장', 88), ('아베_총리', 82), ('사립_유치원', 82), ('공시_가격', 76), ('무역_협상', 73), ('푸틴_대통령', 73), ('김_차관', 73), ('영업_이익', 72), ('증거_인멸', 71), ('패스트_트랙', 70), ('주택_공시', 69), ('차_북미', 67), ('낙태_죄', 60), ('핵_시설', 59), ('마약_투약', 58), ('주주_행사', 55), ('대북_제재', 53), ('남북_정상', 53), ('관세_부과', 51), ('김_장관', 51), ('박_대표', 51), ('국민_투표', 50), ('단독_주택', 49), ('실무_협상', 48), ('경상_수지', 46), ('산하_기관', 45), ('매매_알선', 45), ('한_미', 42), ('탄력_근로', 42), ('사내_이사', 41), ('불법_촬영', 41), ('장관_후보자', 41), ('영변_핵', 40), ('탄도_미사일', 39), ('유도_무기', 39), ('무상_교육', 38), ('비례_대표', 36), ('유_대표', 36), ('지정_취소', 36), ('기자_회견', 35), ('국경_장벽', 35), ('최저_임금', 35), ('보고서_채택', 35), ('차_북', 34), ('국가_비상사태', 34), ('부장_판사', 34), ('고교_무상', 34), ('전술_유도', 33), ('여야_당', 32), ('뮬러_특검', 32), ('통영_고성', 32), ('한미_정상', 31), ('북한_핵

In [22]:
#클러스터별 N GRAM


for i in range(df['labels'].max()+1):
    docs = []
    merge_tokens = []
    print(i, "===================")
    for article in df[df['labels'] == i]['N_list']:
        doc = []
        com = []
        context = []
        merge_token = []

        # str -> list
        for line in article.split('], ['):
            arr = re.sub("[\[,'\]]", '', line).split(' ')
            doc.append(arr)
            com += list(zip(arr, arr[1:]))

        #2 N_GRAM 체크
        z = Counter(com).most_common()
        for item, value in z:
            if value > 3:
                merge_token.append(item[0]+'_'+item[1])
        if len(merge_token) > 0:
            merge_tokens += merge_token

        # 토큰을 합치는 내용
        for token in merge_token:
            for content in doc:
                for i in range(len(content)-1):
                    cmp_str = '_'.join(content[i:i+2])
                    if cmp_str == token:
                        del content[i:i+2]
                        content.insert(i, token)
        docs.append(doc)
    z = Counter(merge_tokens).most_common()
    print(z)

0 ===================
[('정상_회담', 722), ('트럼프_대통령', 620), ('김_위원장', 438), ('북_미', 202), ('문_대통령', 150), ('미_정상', 135), ('북미_정상', 111), ('푸틴_대통령', 73), ('무역_협상', 67), ('차_북미', 62), ('남북_정상', 53), ('관세_부과', 50), ('핵_시설', 48), ('실무_협상', 46), ('한_미', 38), ('대북_제재', 37), ('차_북', 32), ('영변_핵', 31), ('북한_핵', 30), ('차_정상', 28), ('제재_해제', 26), ('주석_방북', 25), ('무역_전쟁', 24), ('핵_협상', 24), ('비건_대표', 24), ('제재_완화', 23), ('정상_회의', 22), ('김정은_위원장', 21), ('미_북', 21), ('한미_정상', 21), ('북_정상', 20), ('미국_대통령', 20), ('볼턴_보좌관', 17), ('관세_인상', 16), ('추가_관세', 16), ('북_러', 16), ('도널드_트럼프', 16), ('회담_결렬', 15), ('위원장_친서', 14), ('베트남_하노이', 14), ('기자_회견', 14), ('단독_회담', 13), ('대통령_김', 13), ('중국_제품', 12), ('미중_무역', 12), ('북_핵', 12), ('대통령_친서', 12), ('미국_중국', 11), ('트럼프_미국', 11), ('협상_재개', 11), ('김_부위원장', 11), ('메트로_폴', 11), ('미_무역', 10), ('대통령_김정은', 10), ('한반도_평화', 10), ('시설_폐기', 10), ('폴_호텔', 10), ('달러_규모', 9), ('이번_회담', 9), ('대통령_방한', 9), ('대화_재개', 9), ('완전_핵', 9), ('상응_조치', 9), ('국무위원_장', 9), ('제품_관세', 8), ('극동_연

In [155]:
from gensim.models import KeyedVectors

loaded_model=KeyedVectors.load("word_vec2")
loaded_model.most_similar("")

[('경협', 0.43442708253860474),
 ('김영철', 0.40458330512046814),
 ('고대', 0.3864108920097351),
 ('매파', 0.3814399540424347),
 ('노골적', 0.38012757897377014),
 ('통일전선부', 0.37808918952941895),
 ('조미수뇌', 0.3741137385368347),
 ('이발', 0.37381061911582947),
 ('차 북미', 0.36483627557754517),
 ('대톨령', 0.3632027804851532)]

In [154]:
loaded_model=KeyedVectors.load("word_fast2")
loaded_model.most_similar("정은")

[('이정은', 0.8745384812355042),
 ('페이지', 0.8439659476280212),
 ('병기', 0.8291098475456238),
 ('정착', 0.819145917892456),
 ('방침', 0.7895593643188477),
 ('디', 0.7649025917053223),
 ('상임', 0.7409874200820923),
 ('김정은', 0.7392602562904358),
 ('국가안전보장회', 0.7311857342720032),
 ('파악', 0.6985346078872681)]

In [98]:
import pandas as pd
from gensim.models import word2vec
from gensim.models import Word2Vec


word_list = []

for i in range(df['labels'].max()+1):
    for article in df[df['labels'] == i]['N_list']:
        doc = []
        for line in article:
            doc += line
            word_list.append(doc)
    
print(len(word_list))
my_model = Word2Vec(word_list, window = 100, min_count = 1, sg = 1,workers=4,vector_size=200)
word_vectors = my_model.wv
word_vectors.save('word_vec2')

81673


In [99]:
import pandas as pd
from gensim.models import word2vec


word_list = []
for article in df[df['labels'] == 36]['N_list']: #번호에 해당 클러스터 번호입력
    doc = []
    for line in article:
        doc += line
        word_list.append(doc)

print(len(word_list))
my_model_fast = FastText(word_list, window = 100, min_count = 1, sg = 1,vector_size=100)

word_vectors_fast = my_model_fast.wv
word_vectors_fast.save('word_fast2')


297
